In [247]:
# !python -m pip uninstall scikit-learn --yes

In [248]:
# pip install scikit-learn==1.2.2 --user

In [249]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [250]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [251]:
# import sklearn
# print(sklearn.__version__)

In [252]:
from imblearn.over_sampling import SMOTENC
from sklearn.utils import resample

In [253]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [254]:
file_path = './data/data_iu_proccessed_p3.csv'
# file_path = './data/data_iu_proccessed_p3_v2.csv'

# Read the CSV file with specified data types
df = pd.read_csv(file_path)

In [255]:
df.head(5)

,Unnamed: 0,TenMH,MaMH,Major,SoTinChi,SiSoTKB,ClassifyAs,TKB_LT2,TKB_LT3,TKB_LT4,...,TKB_LT7,TKB_TH2,TKB_TH3,TKB_TH4,TKB_TH5,TKB_TH6,TKB_TH7,HK,GVLT,GVTH
0,0,Kinh tế hành vi và chính sách công,PM5110,BA BA,2,50.0,4,0,0,0,...,0,0,0,0,0,0,0,2,Nguyen Van Phuong,unassigned
1,1,Kinh tế hành vi và chính sách công,PM5110,BA BA,2,61.0,2,0,0,0,...,0,0,0,0,0,0,0,2,Nguyen Van Phuong,unassigned
2,2,Kinh tế hành vi và chính sách công,PM5110,BA BA,2,14.0,1,0,0,0,...,7,0,0,0,0,0,0,3,Nguyen Van Phuong,unassigned
3,3,Kinh tế lượng và phân tích chính sách,PM5214,BA BA,2,50.0,4,0,0,0,...,0,0,0,0,0,0,0,2,Nguyen Phuong Anh,unassigned
4,4,Kỹ năng lãnh đạo trong khu vực công,PM5212,BA BA,2,41.0,2,0,0,0,...,7,0,0,0,0,0,0,1,Trinh Viet Dung,unassigned


In [256]:
# Remove the specified columns
columns_to_remove = ['Unnamed: 0', 'TenMH']  # Columns to be removed
df.drop(columns=columns_to_remove, inplace=True)
df.dtypes

MaMH           object
Major          object
SoTinChi        int64
SiSoTKB       float64
ClassifyAs      int64
TKB_LT2         int64
TKB_LT3         int64
TKB_LT4         int64
TKB_LT5         int64
TKB_LT6         int64
TKB_LT7         int64
TKB_TH2         int64
TKB_TH3         int64
TKB_TH4         int64
TKB_TH5         int64
TKB_TH6         int64
TKB_TH7         int64
HK              int64
GVLT           object
GVTH           object
dtype: object

In [257]:
df.shape

(8341, 20)

In [2]:
#-----------4 Classes--------------
# #Create a dictionary to map values
# class_mapping = {1: 'Class1', 2: 'Class2', 3: 'Class3', 4: 'Class4'}

# # Map the values in the 'ClassifyAs' column
# df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)

# #-----------2 Classes (Over90)--------------
# Create a dictionary to map values
class_mapping = {1: 'Class1', 2: 'Class1', 3: 'Class2', 4: 'Class2'}

# Map the values in the 'ClassifyAs' column
df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)

#---------- 2 Classes (TR89-Spread)-------------
# # Create a dictionary to map values
# class_mapping = {1: 'Class1', 2: 'Class2', 3: 'Class1', 4: 'Class2'}

# # Map the values in the 'ClassifyAs' column
# df['ClassifyAs'] = df['ClassifyAs'].map(class_mapping)


SyntaxError: invalid syntax (1041017622.py, line 23)

In [259]:
# Count the occurrences of each class
class_counts = df['ClassifyAs'].value_counts()

# Print the counts
print(class_counts)

ClassifyAs
Class2    6562
Class1    1779
Name: count, dtype: int64


In [260]:
#===========================================================================================================
#-------------------------Normal / Class Weight-------------------------------------------------------------
#===========================================================================================================

# Split the data into features (X) and target (y)
X = df.drop(columns=['ClassifyAs'])  # Features
y = df['ClassifyAs']  # Target

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define custom class weights
# Total number of samples
# total_samples = 1050 + 3437 + 751 + 3749

# # Calculate class weights
# weight_class_1 = total_samples / 1050
# weight_class_2 = total_samples / 3437
# weight_class_3 = total_samples / 751
# weight_class_4 = total_samples / 3749

# for classifier use Logloss, CrossEntropy, MultiClass, MultiClassOneVsAll
# Create CatBoost classifier
catboost_clf = CatBoostClassifier(iterations=1000,
                                   depth=4, 
                                   learning_rate=0.2,
#                                    l2_leaf_reg=1,
                                   loss_function='MultiClass',
#                                    loss_function='Logloss',
                                   eval_metric='Accuracy',
                                   cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
                                   auto_class_weights='Balanced',  # Automatically balance class weights
#                                    class_weights=[weight_class_1, weight_class_2, weight_class_3, weight_class_4],  # Manually adjust class weights
                                   random_seed=42)

# Fit the model
catboost_clf.fit(X_train, y_train)

# Predict on the test set
y_pred = catboost_clf.predict(X_test)

accuracy = catboost_clf.score(X_test, y_test)
print("Model Accuracy:", accuracy)

# #-------------Cross Validate-------------------------------------------------------------

# # Create CatBoost classifier
# catboost_clf = CatBoostClassifier(iterations=500,
#                                    depth=6, 
#                                    learning_rate=0.2,
#                                    loss_function='Logloss', 
#                                    eval_metric='Accuracy',
#                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  
#                                    random_seed=42)

# # Perform cross-validation
# cv_scores = cross_val_score(catboost_clf, X_train, y_train, cv=5, scoring='accuracy')  # You can adjust cv (number of folds) as needed

# # Print cross-validation scores
# print("Cross-Validation Scores:", cv_scores)
# print("Mean Accuracy:", np.mean(cv_scores))

#===========================================================================================================
#------------------SMOTE-N----------------------------------------------------------------------------------
#===========================================================================================================

# # Step 1: Split the dataset into features (X) and target variable (y)
# X = df.drop(columns=['ClassifyAs'])  # Features
# y = df['ClassifyAs']  # Target

# # Step 2: Split the features and target into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # Step 3: Apply SMOTE-N to balance the training data
# categorical_indices = [index for index, column in enumerate(X_train.columns) if column in ['MaMH', 'Major', 'GVLT', 'GVTH']]
# smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42)
# X_resampled, y_resampled = smote_nc.fit_resample(X_train, y_train)

# # Step 4: Train the model on the balanced training data
# catboost_clf = CatBoostClassifier(iterations=500,
#                                    depth=6, 
#                                    learning_rate=0.2,
# #                                    loss_function='MultiClass',
#                                    loss_function='Logloss',
#                                    grow_policy='Lossguide',
#                                    eval_metric='Accuracy',
#                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  
#                                    random_seed=42)
# catboost_clf.fit(X_resampled, y_resampled)

# # Step 5: Predict on the testing data
# y_pred = catboost_clf.predict(X_test)

# # Step 6: Evaluate the model
# accuracy = catboost_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

#===========================================================================================================
#------------------Oversampling Minor Class (2 Classes)-----------------------------------------------------
#===========================================================================================================

# # Split the data into features (X) and target (y)
# X = df.drop(columns=['ClassifyAs'])  # Features
# y = df['ClassifyAs']  # Target

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Combine the features and target into a single DataFrame for the training data
# train_data = pd.concat([X_train, y_train], axis=1)

# # Separate majority and minority classes
# majority_class = train_data[train_data['ClassifyAs'] == 'Class2'] 
# minority_class = train_data[train_data['ClassifyAs'] == 'Class1']

# # Upsample minority class
# minority_upsampled = resample(minority_class,
#                               replace=True,     # Sample with replacement
#                               n_samples=len(majority_class),    # Match number of samples in majority class
#                               random_state=42)  # Set random state for reproducibility

# # Combine majority class and upsampled minority class
# train_upsampled = pd.concat([majority_class, minority_upsampled])

# # Separate features and target after oversampling
# X_train_resampled = train_upsampled.drop(columns=['ClassifyAs'])
# y_train_resampled = train_upsampled['ClassifyAs']

# # Create CatBoost classifier with balanced class weights
# catboost_clf = CatBoostClassifier(iterations=5000,
#                                    depth=6, 
#                                    learning_rate=0.1,
#                                    loss_function='Logloss',
#                                    eval_metric='Accuracy',
#                                    cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
#                                    auto_class_weights='Balanced',  # Automatically balance class weights
#                                    random_seed=42)

# # Fit the model on the resampled data
# catboost_clf.fit(X_train_resampled, y_train_resampled)

# # Predict on the test set
# y_pred = catboost_clf.predict(X_test)

# # Evaluate model accuracy
# accuracy = catboost_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

#===========================================================================================================
#------------------Oversampling Minor Class (4 Classes)-----------------------------------------------------
#===========================================================================================================

# # Split the data into features (X) and target (y)
# X = df.drop(columns=['ClassifyAs'])  # Features
# y = df['ClassifyAs']  # Target

# # Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=29)

# # Combine the features and target into a single DataFrame for the training data
# train_data = pd.concat([X_train, y_train], axis=1)

# # Separate majority and minority classes
# majority_class = train_data[train_data['ClassifyAs'] == 'Class4']  # Assuming 'Class4' is the majority class
# minority_classes = [train_data[train_data['ClassifyAs'] == cls] for cls in ['Class1', 'Class2', 'Class3']]

# # Upsample each minority class individually
# minority_upsampled = pd.concat([resample(minority_class, replace=True, n_samples=len(majority_class), random_state=42)
#                                 for minority_class in minority_classes])

# # Combine majority class and upsampled minority classes
# train_upsampled = pd.concat([majority_class] + [minority_upsampled])

# # Separate features and target after oversampling
# X_train_resampled = train_upsampled.drop(columns=['ClassifyAs'])
# y_train_resampled = train_upsampled['ClassifyAs']

# # Create CatBoost classifier with balanced class weights
# catboost_clf = CatBoostClassifier(iterations=5000,
#                                   depth=6,
#                                   learning_rate=0.1,
#                                   loss_function='MultiClass',
#                                   eval_metric='Accuracy',
#                                   cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],  # Specify high cardinality categorical features
#                                   auto_class_weights='Balanced',  # Automatically balance class weights
#                                   random_seed=42)

# # Fit the model on the resampled data
# catboost_clf.fit(X_train_resampled, y_train_resampled)

# # Predict on the test set
# y_pred = catboost_clf.predict(X_test)

# # Evaluate model accuracy
# accuracy = catboost_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

0:	learn: 0.6629523	total: 12.5ms	remaining: 12.5s
1:	learn: 0.6635583	total: 26.7ms	remaining: 13.3s
2:	learn: 0.6633409	total: 40.4ms	remaining: 13.4s
3:	learn: 0.6627952	total: 54ms	remaining: 13.4s
4:	learn: 0.6758600	total: 68.1ms	remaining: 13.6s
5:	learn: 0.6807237	total: 81.2ms	remaining: 13.5s
6:	learn: 0.6818017	total: 94.3ms	remaining: 13.4s
7:	learn: 0.6804781	total: 108ms	remaining: 13.4s
8:	learn: 0.6819995	total: 122ms	remaining: 13.5s
9:	learn: 0.6846943	total: 136ms	remaining: 13.5s
10:	learn: 0.6875527	total: 151ms	remaining: 13.6s
11:	learn: 0.6880984	total: 166ms	remaining: 13.6s
12:	learn: 0.6893606	total: 181ms	remaining: 13.7s
13:	learn: 0.6894152	total: 196ms	remaining: 13.8s
14:	learn: 0.6893195	total: 210ms	remaining: 13.8s
15:	learn: 0.6895925	total: 224ms	remaining: 13.8s
16:	learn: 0.6900495	total: 239ms	remaining: 13.8s
17:	learn: 0.6938291	total: 254ms	remaining: 13.9s
18:	learn: 0.6925055	total: 268ms	remaining: 13.9s
19:	learn: 0.6927374	total: 283ms	re

168:	learn: 0.7495602	total: 2.55s	remaining: 12.5s
169:	learn: 0.7496557	total: 2.56s	remaining: 12.5s
170:	learn: 0.7504812	total: 2.58s	remaining: 12.5s
171:	learn: 0.7506449	total: 2.59s	remaining: 12.5s
172:	learn: 0.7507405	total: 2.6s	remaining: 12.4s
173:	learn: 0.7492532	total: 2.62s	remaining: 12.4s
174:	learn: 0.7498876	total: 2.63s	remaining: 12.4s
175:	learn: 0.7503311	total: 2.65s	remaining: 12.4s
176:	learn: 0.7502356	total: 2.66s	remaining: 12.4s
177:	learn: 0.7494101	total: 2.67s	remaining: 12.3s
178:	learn: 0.7496625	total: 2.69s	remaining: 12.3s
179:	learn: 0.7507064	total: 2.7s	remaining: 12.3s
180:	learn: 0.7509929	total: 2.71s	remaining: 12.3s
181:	learn: 0.7506790	total: 2.73s	remaining: 12.3s
182:	learn: 0.7515660	total: 2.74s	remaining: 12.2s
183:	learn: 0.7511225	total: 2.75s	remaining: 12.2s
184:	learn: 0.7520776	total: 2.77s	remaining: 12.2s
185:	learn: 0.7515727	total: 2.78s	remaining: 12.2s
186:	learn: 0.7509724	total: 2.8s	remaining: 12.2s
187:	learn: 0.7

337:	learn: 0.7779750	total: 4.92s	remaining: 9.63s
338:	learn: 0.7782616	total: 4.93s	remaining: 9.62s
339:	learn: 0.7785481	total: 4.95s	remaining: 9.6s
340:	learn: 0.7788961	total: 4.96s	remaining: 9.59s
341:	learn: 0.7790530	total: 4.97s	remaining: 9.57s
342:	learn: 0.7796875	total: 4.99s	remaining: 9.55s
343:	learn: 0.7798444	total: 5s	remaining: 9.54s
344:	learn: 0.7791485	total: 5.01s	remaining: 9.52s
345:	learn: 0.7788346	total: 5.03s	remaining: 9.5s
346:	learn: 0.7788346	total: 5.04s	remaining: 9.49s
347:	learn: 0.7791212	total: 5.05s	remaining: 9.47s
348:	learn: 0.7790257	total: 5.07s	remaining: 9.46s
349:	learn: 0.7790257	total: 5.08s	remaining: 9.44s
350:	learn: 0.7791212	total: 5.1s	remaining: 9.42s
351:	learn: 0.7782343	total: 5.11s	remaining: 9.41s
352:	learn: 0.7783912	total: 5.12s	remaining: 9.39s
353:	learn: 0.7801036	total: 5.14s	remaining: 9.38s
354:	learn: 0.7793122	total: 5.15s	remaining: 9.36s
355:	learn: 0.7814340	total: 5.17s	remaining: 9.35s
356:	learn: 0.7821

505:	learn: 0.7999430	total: 7.28s	remaining: 7.11s
506:	learn: 0.8004205	total: 7.3s	remaining: 7.09s
507:	learn: 0.8006116	total: 7.31s	remaining: 7.08s
508:	learn: 0.8002295	total: 7.32s	remaining: 7.06s
509:	learn: 0.8003864	total: 7.34s	remaining: 7.05s
510:	learn: 0.8002295	total: 7.35s	remaining: 7.03s
511:	learn: 0.8007685	total: 7.36s	remaining: 7.02s
512:	learn: 0.8009595	total: 7.38s	remaining: 7s
513:	learn: 0.8016554	total: 7.39s	remaining: 6.99s
514:	learn: 0.8015940	total: 7.41s	remaining: 6.98s
515:	learn: 0.8018805	total: 7.42s	remaining: 6.96s
516:	learn: 0.8019760	total: 7.44s	remaining: 6.95s
517:	learn: 0.8022626	total: 7.45s	remaining: 6.93s
518:	learn: 0.8030881	total: 7.46s	remaining: 6.92s
519:	learn: 0.8028015	total: 7.48s	remaining: 6.9s
520:	learn: 0.8028970	total: 7.49s	remaining: 6.89s
521:	learn: 0.8028356	total: 7.51s	remaining: 6.88s
522:	learn: 0.8027401	total: 7.52s	remaining: 6.86s
523:	learn: 0.8030607	total: 7.54s	remaining: 6.85s
524:	learn: 0.802

668:	learn: 0.8184246	total: 9.66s	remaining: 4.78s
669:	learn: 0.8186157	total: 9.68s	remaining: 4.77s
670:	learn: 0.8185202	total: 9.7s	remaining: 4.75s
671:	learn: 0.8184587	total: 9.71s	remaining: 4.74s
672:	learn: 0.8192501	total: 9.73s	remaining: 4.73s
673:	learn: 0.8184587	total: 9.75s	remaining: 4.71s
674:	learn: 0.8187453	total: 9.76s	remaining: 4.7s
675:	learn: 0.8188408	total: 9.78s	remaining: 4.68s
676:	learn: 0.8190318	total: 9.79s	remaining: 4.67s
677:	learn: 0.8192228	total: 9.8s	remaining: 4.66s
678:	learn: 0.8184314	total: 9.82s	remaining: 4.64s
679:	learn: 0.8197618	total: 9.84s	remaining: 4.63s
680:	learn: 0.8193183	total: 9.85s	remaining: 4.62s
681:	learn: 0.8191614	total: 9.87s	remaining: 4.6s
682:	learn: 0.8195094	total: 9.88s	remaining: 4.59s
683:	learn: 0.8197959	total: 9.9s	remaining: 4.57s
684:	learn: 0.8197004	total: 9.92s	remaining: 4.56s
685:	learn: 0.8192228	total: 9.94s	remaining: 4.55s
686:	learn: 0.8200824	total: 9.95s	remaining: 4.53s
687:	learn: 0.820

834:	learn: 0.8338294	total: 12.5s	remaining: 2.46s
835:	learn: 0.8339249	total: 12.5s	remaining: 2.45s
836:	learn: 0.8339249	total: 12.5s	remaining: 2.44s
837:	learn: 0.8335770	total: 12.5s	remaining: 2.42s
838:	learn: 0.8331335	total: 12.5s	remaining: 2.41s
839:	learn: 0.8342115	total: 12.6s	remaining: 2.39s
840:	learn: 0.8329425	total: 12.6s	remaining: 2.38s
841:	learn: 0.8328470	total: 12.6s	remaining: 2.36s
842:	learn: 0.8325946	total: 12.6s	remaining: 2.35s
843:	learn: 0.8340205	total: 12.6s	remaining: 2.33s
844:	learn: 0.8340205	total: 12.6s	remaining: 2.32s
845:	learn: 0.8345594	total: 12.7s	remaining: 2.3s
846:	learn: 0.8346549	total: 12.7s	remaining: 2.29s
847:	learn: 0.8339249	total: 12.7s	remaining: 2.27s
848:	learn: 0.8339249	total: 12.7s	remaining: 2.26s
849:	learn: 0.8336725	total: 12.7s	remaining: 2.25s
850:	learn: 0.8326287	total: 12.7s	remaining: 2.23s
851:	learn: 0.8333587	total: 12.8s	remaining: 2.22s
852:	learn: 0.8334883	total: 12.8s	remaining: 2.2s
853:	learn: 0.

994:	learn: 0.8464780	total: 15.3s	remaining: 76.8ms
995:	learn: 0.8464780	total: 15.3s	remaining: 61.4ms
996:	learn: 0.8464780	total: 15.3s	remaining: 46.1ms
997:	learn: 0.8471124	total: 15.3s	remaining: 30.7ms
998:	learn: 0.8469555	total: 15.3s	remaining: 15.4ms
999:	learn: 0.8469555	total: 15.4s	remaining: 0us
Model Accuracy: 0.7297783103654883


In [261]:
X_test.head(5)

,MaMH,Major,SoTinChi,SiSoTKB,TKB_LT2,TKB_LT3,TKB_LT4,TKB_LT5,TKB_LT6,TKB_LT7,TKB_TH2,TKB_TH3,TKB_TH4,TKB_TH5,TKB_TH6,TKB_TH7,HK,GVLT,GVTH
7023,PE017IU,IU PE,2,90.0,0,0,0,0,0,3,0,0,0,0,0,0,1,Pham Dinh Huan,unassigned
5201,MAFE208IU,MA MA,4,50.0,7,0,0,0,0,0,0,0,0,0,0,0,2,Mai Duc Thanh,unassigned
4280,IS056IU,IEMIEM,1,25.0,0,0,0,0,0,0,0,0,0,0,7,0,2,unassigned,Nguyen Hoang An
940,CH012IU,BT BT,1,20.0,7,0,0,0,0,0,0,0,0,0,0,0,3,Le Nguyen Thien Phuc,unassigned
5364,BA130IU,BA BA,3,60.0,0,0,0,0,0,1,0,0,0,0,0,0,1,Nguyen Tran Nguyen Khai,unassigned


In [262]:
X_test.dtypes

MaMH         object
Major        object
SoTinChi      int64
SiSoTKB     float64
TKB_LT2       int64
TKB_LT3       int64
TKB_LT4       int64
TKB_LT5       int64
TKB_LT6       int64
TKB_LT7       int64
TKB_TH2       int64
TKB_TH3       int64
TKB_TH4       int64
TKB_TH5       int64
TKB_TH6       int64
TKB_TH7       int64
HK            int64
GVLT         object
GVTH         object
dtype: object

In [263]:
X_test.shape

(1669, 19)

In [264]:
# # Define the parameter grid
# param_grid = {
#     'iterations': [500, 1000],  # Number of boosting iterations
#     'depth': [4, 6, 8],          # Maximum depth of each tree
#     'learning_rate': [0.05, 0.1],# Learning rate
#     # Add more parameters to tune if needed
# }

# # Create CatBoost classifier
# catboost_clf = CatBoostClassifier(loss_function='Logloss',
#                                   eval_metric='Accuracy',
#                                   cat_features=['MaMH', 'Major', 'GVLT', 'GVTH'],
#                                   auto_class_weights='Balanced',
#                                   random_seed=42)

# # Perform grid search
# grid_search = GridSearchCV(estimator=catboost_clf, param_grid=param_grid, cv=5, scoring='accuracy')
# grid_search.fit(X_train, y_train)

# # Best parameters found
# print("Best parameters:", grid_search.best_params_)

# # Best score
# print("Best accuracy:", grid_search.best_score_)

# # Predict on the test set using the best model
# best_clf = grid_search.best_estimator_
# y_pred = best_clf.predict(X_test)

# # Calculate accuracy
# accuracy = best_clf.score(X_test, y_test)
# print("Model Accuracy:", accuracy)

In [265]:
# # Convert y_resampled to a pandas Series
y_train_series = pd.Series(y_train)

# Count the occurrences of each class
class_counts2 = y_train_series.value_counts()

# Print the class counts
print("Class Counts in y_train_series:")
print(class_counts2)

Class Counts in y_train_series:
ClassifyAs
Class2    5235
Class1    1437
Name: count, dtype: int64


In [266]:
# # Convert y_resampled to a pandas Series
# # y_resampled_series = pd.Series(y_resampled)
# y_resampled_series = pd.Series(y_train_resampled)

# # Count the occurrences of each class
# class_counts = y_resampled_series.value_counts()

# # Print the class counts
# print("Class Counts in y_resampled:")
# print(class_counts)

In [267]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Create a DataFrame for actual vs. predicted values
results_df = pd.DataFrame({'Actual': y_test.values.flatten(), 'Predicted': y_pred.flatten()})
results_df

Accuracy: 0.7297783103654883


,Actual,Predicted
0,Class2,Class2
1,Class2,Class2
2,Class2,Class2
3,Class2,Class1
4,Class2,Class2
5,Class2,Class2
6,Class2,Class2
7,Class2,Class2
8,Class2,Class2
9,Class1,Class1


In [268]:
# Create a DataFrame to count total correct and incorrect predictions for each class
class_counts = {}

# Update counts based on actual vs. predicted values
for actual, predicted in zip(y_test, y_pred):
    if actual not in class_counts:
        class_counts[actual] = {'Correct': 0, 'Incorrect': 0}
    if actual == predicted:
        class_counts[actual]['Correct'] += 1
    else:
        class_counts[actual]['Incorrect'] += 1

# Convert dictionary to DataFrame
class_count_df = pd.DataFrame(class_counts).transpose().reset_index()
class_count_df.rename(columns={'index': 'Class'}, inplace=True)

# Calculate total correct and incorrect predictions across all classes
total_correct_all_classes = class_count_df['Correct'].sum()
total_incorrect_all_classes = class_count_df['Incorrect'].sum()

# Add a row for the total counts across all classes
total_row = pd.DataFrame({'Class': 'Total', 'Correct': total_correct_all_classes, 'Incorrect': total_incorrect_all_classes}, index=[0])
class_count_df = pd.concat([class_count_df, total_row], ignore_index=True)

# Display the DataFrame
print("\nClass-wise Correct and Incorrect Predictions:")
print(class_count_df)


Class-wise Correct and Incorrect Predictions:
    Class  Correct  Incorrect
0  Class2      980        347
1  Class1      238        104
2   Total     1218        451


In [269]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# # Classification Report
# print("\nClassification Report:")
print(classification_report(y_test, y_pred))

Confusion Matrix:
[[238 104]
 [347 980]]
              precision    recall  f1-score   support

      Class1       0.41      0.70      0.51       342
      Class2       0.90      0.74      0.81      1327

    accuracy                           0.73      1669
   macro avg       0.66      0.72      0.66      1669
weighted avg       0.80      0.73      0.75      1669



In [270]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.7297783103654883
Precision: 0.8021718444589155
Recall: 0.7297783103654883
F1-score: 0.751578157063824


In [271]:
# # Define the filename to save your model
# model_filename = 'dhqt_90slots_classifier_model.cbm'
# # model_filename = 'dhqt_tr89_classifier_model.cbm'

# # Save the model
# catboost_clf.save_model(model_filename)

# print("Model saved successfully as", model_filename)